In [54]:
from vec import Vec

In [55]:
id_movies = {(i.strip().split(','))[0] for i in list(open('movies.csv'))}
id_user = {(i.strip().split(','))[0] for i in list(open('ratings.csv'))}
id_user.remove('user_id')
users = {}

for i in id_user:
    dic = {}
    archivo = open('ratings.csv', 'r')
    archivo = archivo.readlines()
    
    for x in archivo:
        x = x.strip('\n')
        x = x.split(',')
        if x[0] == i:
            dic[x[1]] = int(x[2]) 
    users[i] = Vec(id_movies,dic)

In [56]:
users['943']['2']


5

In [57]:
id_movies.remove('movie_id')
movies = {}

for i in id_movies:
    dic = {}
    archivo = open('ratings.csv', 'r')
    archivo = archivo.readlines()
    
    for x in archivo:
        x = x.strip('\n')
        x = x.split(',')
        if x[1] == i:
            dic[x[0]] = int(x[2]) 
    movies[i] = Vec(id_user,dic)

In [58]:
movies['1']['1']

0